# Scrape IMDB to match movie IDs with movie titles

In [1]:
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup
import re
import sys
sys.path.append("../movielingo/")
%load_ext autoreload
%autoreload 2
import time
import requests
import json
import tqdm

In [33]:
def get_movie_characteristics(imdb_id):
    url = 'https://www.imdb.com/title/tt' + imdb_id + '/'
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    title = html_soup.title.text
    description = html_soup.find('script', type="application/ld+json").contents[0]
    description = json.loads(description)
    genre = description['genre']
    movie_or_tv = description['@type']
    keywords = description['keywords']
    rating = description['aggregateRating']['ratingValue']
    return title, genre, movie_or_tv, keywords, rating

In [3]:
ids = []
for file in files:
    filename = os.path.join(directory+file)
    imdb_id = re.search("(\d{7})", file).group()
    ids.append(imdb_id)

In [ ]:
movie_titles = []
unique_ids = list(set(ids))

for imdb_id in tqdm.tqdm(unique_ids):
    try:
        title, genre, movie_or_tv, keywords, rating = get_movie_characteristics(imdb_id)
        time.sleep(5)
        with open('movie_characteristics.txt', 'a') as movie_characteristics_file:
            movie_characteristics_file.write(imdb_id + '\t' + 
                                      title + '\t' + 
                                      genre + '\t' +
                                      movie_or_tv + '\t' +
                                      keywords + '\t' +
                                      rating +
                                      '\n')
    except:
        pass

 34%|███▍      | 4295/12581 [6:45:36<11:38:58,  5.06s/it]

In [61]:
imdb = pd.read_csv('/Users/aglushko/Desktop/insight_fellows/insight-project/insight-data-project/notebooks/movie_characteristics.txt', sep = '\t', names = ['id', 'title', 'genre', 'movie_or_show', 'keywords', 'rating'])

In [62]:
imdb['rating'] = imdb.rating.astype(float)
imdb['id'] = imdb.id.astype(str)

In [73]:
imdb_good = imdb[imdb.rating > 7].reset_index(drop=True).copy()

In [74]:
imdb_good

,id,title,genre,movie_or_show,keywords,rating
0,67484,Nie lubie poniedzialku (1971) - IMDb,Comedy,Movie,"italian,statement in title,monday,thief,robbery",7.6
1,109686,Dumb and Dumber (1994) - IMDb,Comedy,Movie,"road trip,briefcase of money,woman wearing a s...",7.3
2,112220,The Wayans Bros. (TV Series 1995–1999) - IMDb,Comedy,TVSeries,"three word title,title ends with period,period...",7.2
3,47238,Miseria e nobiltà (1954) - IMDb,Comedy,Movie,"false identity,italy,spaghetti,pasta,poverty",7.8
4,268077,That's My Bush! (TV Series 2001) - IMDb,Comedy,TVSeries,"american politics,political leader,republican,...",7.3
...,...,...,...,...,...,...
137,414773,The Smoking Room (TV Series 2004–2005) - IMDb,Comedy,TVSeries,"sitcom,british comedy,smoking room",7.9
138,460091,My Name Is Earl (TV Series 2005–2009) - IMDb,Comedy,TVSeries,"heavy metal,crab,karma,list,low life",7.7
139,795156,Monty Python's Personal Best (TV Series 2006) ...,Comedy,TVSeries,"compilation,flail,sketch comedy,comedy troupe,...",8.0
140,81590,Sällskapsresan eller Finns det svenskt kaffe p...,Comedy,Movie,"or as title separator,first part,part of serie...",7.2


In [75]:
imdb_good.loc[:,'title'] = imdb_good.title.str.split(' \(.{4,40}\) - IMDb', expand=True)[0]

In [76]:
from movielingo.config import processed_data_dir
imdb_good.to_csv(processed_data_dir / 'imdb_title_and_id_matches.csv', index = False)

In [77]:
IMDB = pd.read_csv(processed_data_dir / 'imdb_title_and_id_matches.csv')
IMDB.head()

,id,title,genre,movie_or_show,keywords,rating
0,67484,Nie lubie poniedzialku,Comedy,Movie,"italian,statement in title,monday,thief,robbery",7.6
1,109686,Dumb and Dumber,Comedy,Movie,"road trip,briefcase of money,woman wearing a s...",7.3
2,112220,The Wayans Bros.,Comedy,TVSeries,"three word title,title ends with period,period...",7.2
3,47238,Miseria e nobiltà,Comedy,Movie,"false identity,italy,spaghetti,pasta,poverty",7.8
4,268077,That's My Bush!,Comedy,TVSeries,"american politics,political leader,republican,...",7.3
